In [3]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import itertools
import csv
import warnings
import pickle
import scipy

import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


In [4]:
Y_train = pd.read_csv("data/cdiscount_train_subset.csv").fillna("")["Categorie1"]
Y_test = pd.read_csv("data/cdiscount_test.csv").fillna("")["Categorie1"]

# Regression Logistique

In [3]:
DATA_DIR = "data/features"

from scipy import sparse

metadata_list_lr = []

parameters = [[None, "count"],
              [300, "count"],
              [10000, "count"],
              [None, "tfidf"],
              [300, "tfidf"],
              [10000, "tfidf"],]

for nb_hash, vectorizer in parameters:
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    X_train = sparse.load_npz(DATA_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    X_test = sparse.load_npz(DATA_DIR +"/vec_test_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    ts = time.time()
    cla = LogisticRegression()
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": None, "nb_hash": nb_hash, "vectorizer":vectorizer , "learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_lr.append(metadata)
       

nb_hash : None, vectorizer : count
{'typeW2V': None, 'nb_hash': None, 'vectorizer': 'count', 'learning_time': 65.79484820365906, 'predict_time': 0.20164084434509277, 'score_train': 0.9799368421052631, 'score_valid': 0.9066}
nb_hash : 300, vectorizer : count
{'typeW2V': None, 'nb_hash': 300, 'vectorizer': 'count', 'learning_time': 57.50672173500061, 'predict_time': 0.21946358680725098, 'score_train': 0.7234, 'score_valid': 0.7076}
nb_hash : 10000, vectorizer : count
{'typeW2V': None, 'nb_hash': 10000, 'vectorizer': 'count', 'learning_time': 69.89495491981506, 'predict_time': 0.18376874923706055, 'score_train': 0.9694, 'score_valid': 0.888}
nb_hash : None, vectorizer : tfidf
{'typeW2V': None, 'nb_hash': None, 'vectorizer': 'tfidf', 'learning_time': 35.86675786972046, 'predict_time': 0.20187044143676758, 'score_train': 0.9134736842105263, 'score_valid': 0.8824}
nb_hash : 300, vectorizer : tfidf
{'typeW2V': None, 'nb_hash': 300, 'vectorizer': 'tfidf', 'learning_time': 36.75836682319641, 'p

In [4]:
# Temps d'exécution un peu long....

for model_name in ["CBOW","skip-gram", "online"]:
    print("Word2Vec :" + model_name)

    X_train = np.load(DATA_DIR +"/embedded_train_nb_hash_" + model_name+".npy")
    X_test = np.load(DATA_DIR +"/embedded_test_nb_hash_" + model_name+".npy")
    
    ts = time.time()
    cla = LogisticRegression()
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": model_name ,"nb_hash": None, "vectorizer":"word2vec" ,"learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_lr.append(metadata)



Word2Vec :CBOW
{'typeW2V': 'CBOW', 'nb_hash': None, 'vectorizer': 'word2vec', 'learning_time': 744.2787802219391, 'predict_time': 0.21297192573547363, 'score_train': 0.8030842105263158, 'score_valid': 0.8026}
Word2Vec :skip-gram
{'typeW2V': 'skip-gram', 'nb_hash': None, 'vectorizer': 'word2vec', 'learning_time': 364.36927318573, 'predict_time': 0.21204614639282227, 'score_train': 0.8490736842105263, 'score_valid': 0.843}
Word2Vec :online
{'typeW2V': 'online', 'nb_hash': None, 'vectorizer': 'word2vec', 'learning_time': 757.8160936832428, 'predict_time': 0.18524646759033203, 'score_train': 0.7748, 'score_valid': 0.7656}


In [7]:
np.load("data/features/embedded_train_nb_hash_CBOW.npy").shape

(95000, 300)

In [5]:
pd.DataFrame(metadata_list_lr)

,learning_time,nb_hash,predict_time,score_train,score_valid,typeW2V,vectorizer
0,65.794848,NaN,0.201641,0.979937,0.9066,None,count
1,57.506722,300.0,0.219464,0.723400,0.7076,None,count
2,69.894955,10000.0,0.183769,0.969400,0.8880,None,count
3,35.866758,NaN,0.201870,0.913474,0.8824,None,tfidf
4,36.758367,300.0,0.151250,0.716905,0.7004,None,tfidf
5,32.383074,10000.0,0.187656,0.901989,0.8648,None,tfidf
6,744.278780,NaN,0.212972,0.803084,0.8026,CBOW,word2vec
7,364.369273,NaN,0.212046,0.849074,0.8430,skip-gram,word2vec
8,757.816094,NaN,0.185246,0.774800,0.7656,online,word2vec


# Random Forest

In [6]:
from sklearn.ensemble import RandomForestClassifier
metadata_list_rf = []

parameters = [[None, "count"],
              [300, "count"],
              [10000, "count"],
              [None, "tfidf"],
              [300, "tfidf"],
              [10000, "tfidf"],]

for nb_hash, vectorizer in parameters:
    print("nb_hash : " + str(nb_hash) + ", vectorizer : " + str(vectorizer))
    X_train = sparse.load_npz(DATA_DIR +"/vec_train_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    X_test = sparse.load_npz(DATA_DIR +"/vec_test_nb_hash_" + str(nb_hash) + "_vectorizer_" + str(vectorizer)+".npz")
    ts = time.time()
    cla = RandomForestClassifier(n_estimators=100)
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": None, "nb_hash": nb_hash, "vectorizer":vectorizer , "learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_rf.append(metadata)

nb_hash : None, vectorizer : count
{'typeW2V': None, 'nb_hash': None, 'vectorizer': 'count', 'learning_time': 385.3659739494324, 'predict_time': 11.882237911224365, 'score_train': 0.9986421052631579, 'score_valid': 0.8558}
nb_hash : 300, vectorizer : count
{'typeW2V': None, 'nb_hash': 300, 'vectorizer': 'count', 'learning_time': 250.5232491493225, 'predict_time': 7.298325300216675, 'score_train': 0.9983578947368421, 'score_valid': 0.7726}
nb_hash : 10000, vectorizer : count
{'typeW2V': None, 'nb_hash': 10000, 'vectorizer': 'count', 'learning_time': 201.2321469783783, 'predict_time': 13.452975511550903, 'score_train': 0.9986421052631579, 'score_valid': 0.8484}
nb_hash : None, vectorizer : tfidf
{'typeW2V': None, 'nb_hash': None, 'vectorizer': 'tfidf', 'learning_time': 292.9067871570587, 'predict_time': 10.577593326568604, 'score_train': 0.9986421052631579, 'score_valid': 0.8522}
nb_hash : 300, vectorizer : tfidf
{'typeW2V': None, 'nb_hash': 300, 'vectorizer': 'tfidf', 'learning_time': 2

In [7]:
print("")

for model_name in ["CBOW","skip-gram", "online"]:
    print("Word2Vec :" + model_name)

    X_train = np.load(DATA_DIR +"/embedded_train_nb_hash_" + model_name+".npy")
    X_test = np.load(DATA_DIR +"/embedded_test_nb_hash_" + model_name+".npy")
    
    ts = time.time()
    cla = RandomForestClassifier(n_estimators=100)
    cla.fit(X_train,Y_train.values)
    te=time.time()
    t_learning = te-ts
    ts = time.time()
    score_train=cla.score(X_train,Y_train)
    score_test=cla.score(X_test,Y_test)
    te=time.time()
    t_predict = te-ts
    metadata = {"typeW2V": model_name ,"nb_hash": None, "vectorizer":"word2vec" ,"learning_time" : t_learning, "predict_time":t_predict, "score_train": score_train, "score_test": score_test}
    print(metadata)
    metadata_list_rf.append(metadata)



Word2Vec :CBOW
{'typeW2V': 'CBOW', 'nb_hash': None, 'vectorizer': 'word2vec', 'learning_time': 330.7274811267853, 'predict_time': 5.565982818603516, 'score_train': 0.9986421052631579, 'score_valid': 0.8324}
Word2Vec :skip-gram
{'typeW2V': 'skip-gram', 'nb_hash': None, 'vectorizer': 'word2vec', 'learning_time': 329.91056537628174, 'predict_time': 5.531147241592407, 'score_train': 0.9986421052631579, 'score_valid': 0.8536}
Word2Vec :online
{'typeW2V': 'online', 'nb_hash': None, 'vectorizer': 'word2vec', 'learning_time': 287.7881805896759, 'predict_time': 6.184351205825806, 'score_train': 0.9911368421052632, 'score_valid': 0.7408}


In [8]:
pd.DataFrame(metadata_list_lr)

,learning_time,nb_hash,predict_time,score_train,score_valid,typeW2V,vectorizer
0,65.794848,NaN,0.201641,0.979937,0.9066,None,count
1,57.506722,300.0,0.219464,0.723400,0.7076,None,count
2,69.894955,10000.0,0.183769,0.969400,0.8880,None,count
3,35.866758,NaN,0.201870,0.913474,0.8824,None,tfidf
4,36.758367,300.0,0.151250,0.716905,0.7004,None,tfidf
5,32.383074,10000.0,0.187656,0.901989,0.8648,None,tfidf
6,744.278780,NaN,0.212972,0.803084,0.8026,CBOW,word2vec
7,364.369273,NaN,0.212046,0.849074,0.8430,skip-gram,word2vec
8,757.816094,NaN,0.185246,0.774800,0.7656,online,word2vec
